# Importing necessary modules

In [2]:
!pip install pycontractions
# !pip install contractions

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

import pandas as pd 
import numpy as np 

from nltk.tokenize import TweetTokenizer
# import contractions
from pycontractions import Contractions
import re as re
import string
import random  

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential 
from keras.layers import Dense, Embedding, Flatten 
from keras.datasets import imdb 
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
# Metrics
from sklearn import utils
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, \
        f1_score, precision_score, recall_score
from sklearn.model_selection import RepeatedKFold
from sklearn.base import clone
import matplotlib.pyplot as plt




from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


  ERROR: Failed building wheel for language-check
  Running setup.py clean for language-check
Failed to build language-check
    Running setup.py install for language-check ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-pq4sb0n0/language-check/setup.py'"'"'; __file__='"'"'/tmp/pip-install-pq4sb0n0/language-check/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-qbl4y4bp/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Removing stop words 

In [ ]:
stop_words = list(stopwords.words('english'))

# depressed persons tend to use first-person pronouns more, and third-person pronouns less. these words might provide indication
excluded_pronouns = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll",
                    "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's",
                     'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves']

for pronoun in excluded_pronouns:
    stop_words.remove(pronoun)

In [ ]:
transcript_filepath = "raw_compiled_transcripts.csv"
df_transcript = pd.read_csv(transcript_filepath, index_col = "Participant_ID")

column_list = df_transcript.columns.tolist()
print(column_list)

df_transcript.head(10)

['Transcript', 'PHQ_Score', 'PHQ_Binary']


,Transcript,PHQ_Score,PHQ_Binary
Participant_ID,,,
300,good atlanta georgia um my parents are from he...,2,0
301,thank you mmm k i'm doing good thank you i'm f...,3,0
302,i'm fine how about yourself i'm from los ange...,4,0
303,okay how 'bout yourself here in california yea...,0,0
304,i'm doing good um from los angeles california ...,6,0
305,i'm doing alright uh originally i'm from calif...,7,0
306,fine uh colorado mhm uh career career possibil...,0,0
307,<laughter> um moscow um my family moved to the...,4,0
308,los angeles california yes um the southern cal...,22,1


In [ ]:
def filter_text(text):
    
    # text = re.sub('<[^<]+?>', '', text) # remove anything enclosed by tags e.g. <>
    # text = re.sub('\[(.*?)\]', '', text) # remove anything enclosed by closed brackets i.e. []
    text = contractions.fix(text) # expands contractions e.g.'he's happy' -> 'he is happy'
    
    text = text.lower() # contractions will capitalize 'i'
    # text = text.translate(str.maketrans("", "", string.punctuation)) # remove remaining punctuations e.g. apostrophe
    
    tokens = TweetTokenizer().tokenize(text) # use TweetTokenizer as transcripts contain informal texts
    
    filtered_sentence = [w for w in tokens if w not in stop_words]
    text = ' '.join(filtered_sentence)
    
    return text

In [ ]:
# Filter the text

original_text = df_transcript.Transcript.iloc[30]

df_transcript.Transcript = df_transcript.Transcript.apply(filter_text)
filtered_text = df_transcript.Transcript.iloc[30]

print(f"The original text is:\n{original_text}")
print("-------------------------------------------")
print(f"The filtered text is:\n{filtered_text}")

NameError: ignored

# Counting different features

## Tagged
We need to find actions that are tagged.

In [ ]:
import re

def find_angle_bracket(transcript): 
  # for word in transcript:
    # print(word)
    pattern = "<(.*?)>"

    substring = re.search(pattern, transcript)

    approved = ["laughter", "clears throat", "sigh" ,"yawn", "sniff", "sniffle",
                "cough", "sharp inhale", "sharp inhaling", "sneeze", "deep breath"]

    if substring == None:
      return "None"
    else: 
      x = substring.group(1).strip() 
      if x not in approved:
        return "None"
      else:     
        return x

In [ ]:
def find_square_bracket(transcript): 
  pattern = "\[(.*?)\]"

  substring = re.search(pattern, transcript)

  #remove all spaces so all input is constant
  approved = ["laughter", "clears throat", "sigh" ,"yawn", "sniff", "sniffle",
              "cough", "sharp inhale", "sharp inhaling", "sneeze", "deep breath"]

  if substring == None:
    return "None"
  else: 
    x = substring.group(1).strip() 
    # print(x)
    if x not in approved:
      return "None"
    else:     
      return (x)

## Pronouns

In [ ]:
def calculating_score(text, target_words):
    score = 0
    for w in text:
        for x in target_words:
            if w == x:
                score += 1
    return score

## Putting Everything Together

In [ ]:
# iterate through raw_compiled_scripts and add count 

def count_features():

  # creating new dataframe output 
  columns = ["Participant_ID", "Laughs", "Sighs", "Clears_Throat", "Yawns", "Sniffs", "Coughs", "Sharp_Inhales", "Sneezes", "Deep_Breaths", 
             "Absolute_Words", "First_Pronoun_Singular", "First_Pronoun_Plural", "Third_Pronoun"]
  df_features = pd.DataFrame(columns=columns)
  # df_features.head()  

  ps = PorterStemmer()

  absolute_list = ["complet", "never", "absolut", "onli", "everi", "not" "alway", "just", "none"]
  first_pronoun_s_list = ["i", "me", "mine", "myself"]
  first_pronoun_p_list = ["we", "us"]
  third_pronoun_list = ["he", "she", "it", "they", "him", "her", "it", "your", "his", "her"]

  #iterate through all words in transcript and count features 
  for index, row in df_transcript.iterrows():
      participant_ID = index
      phq_Binary = row["PHQ_Binary"]
      transcript = row["Transcript"]

      transcript = transcript.split()

      if index == 300:
        print(transcript)

      if index == 333:
        print(transcript)
    
      laughs = 0
      sighs = 0 
      clears_throat = 0 
      yawns = 0 
      sniffs = 0
      coughs = 0
      sharp_inhales = 0 
      sneezes = 0
      deep_breaths = 0 

      absolute_words = 0
      first_pronouns_s = 0
      first_pronouns_p = 0
      third_pronouns = 0

      #calculating pronouns:
      absolute_words = calculating_score(transcript, absolute_list)
      first_pronouns_s = calculating_score(transcript, first_pronoun_s_list)
      first_pronouns_p = calculating_score(transcript, first_pronoun_p_list)
      third_pronouns = calculating_score(transcript, third_pronoun_list)

      for word in transcript: 
        word_angle = find_angle_bracket(word)
        word_square = find_square_bracket(word) 

        # get how many actions 1 person did
        if word_angle != "None" and word_square == "None":
          if word_angle == "laughter":
            laughs += 1
          elif word_angle == "clears throat":
            clears_throat += 1
          elif word_angle == "sigh": 
            sighs += 1
          elif word_angle == "yawn":
            yawns += 1
          elif word_angle == "sniff":
            sniffs += 1
          elif word_angle == "sniffle":
            sniffs += 1
          elif word_angle == "cough":
            coughs += 1
          elif word_angle == "sharp inhale":
            sharp_inhales += 1
          elif word_angle == "sharp inhaling":
            sharp_inhales += 1
          elif word_angle == "sneeze":
            sneezes += 1
          elif word_angle == "deep breath":
            deep_breaths += 1
        
        elif word_square != "None" and word_angle == "None":
          if word_square == "laughter":
            laughs += 1
          elif word_square == "clears throat":
            clears_throat += 1
          elif word_square == "sigh": 
            sighs += 1
          elif word_square == "yawn":
            yawns += 1
          elif word_square == "sniff":
            sniffs += 1
          elif word_square == "sniffle":
            sniffs += 1
          elif word_square == "cough":
            coughs += 1
          elif word_square == "sharp inhale":
            sharp_inhales += 1
          elif word_square == "sharp inhaling":
            sharp_inhales += 1
          elif word_square == "sneeze":
            sneezes += 1
          elif word_square == "deep breath":
            deep_breaths += 1

        # counting the number of pronouns 

        stemmed = ps.stem(word)

        # if stemmed in absolute_list: 
        
        # if stemmed in first_pronoun_s_list:
        #   first_pronouns_s += 1
        # elif stemmed in first_pronoun_p_list:
        #   first_pronouns_p += 1
        # elif stemmed in third_pronoun: 
        #   third_pronouns += 1
    
      new_row = {
                "Participant_ID": index,
                "Laughs": laughs,
                "Sighs": sighs,
                "Clears_Throat": clears_throat,
                "Yawns": yawns,
                "Sniffs": sniffs,
                "Coughs": coughs,
                "Sharp_Inhales": sharp_inhales,
                "Sneezes": sneezes,
                "Deep_Breaths": deep_breaths,
                "Absolute_Words": absolute_words,
                "First_Pronoun_Singular": first_pronouns_s,
                "First_Pronoun_Plural": first_pronouns_p,
                "Third_Pronoun": third_pronouns,
                # "PHQ_Binary": phq_Binary
      }

      # print(new_row)

      df_features = df_features.append(new_row, ignore_index = True)
  
  return df_features

df_features = count_features() 
print(df_features.columns.tolist())
df_features.head(30)


# Creating Sample

## Normal Sample

In [ ]:
np_features = df_features.to_numpy()
print(np_features.shape)
print(np_features)

In [ ]:
def train_test(X, y, testfile):
    test_participants = pd.read_csv(testfile)['participant_ID'].values
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    
    for i in range(y.shape[0]):
        participant_no = y.index[i]
        
        if participant_no in test_participants:
            X_test.append(X[i])
            y_test.append(y[participant_no])
        
        else:
            X_train.append(X[i])
            y_train.append(y[participant_no])
            
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [ ]:
testfile = './test_split_Depression_AVEC2017 (3).csv'

X = np_features
y =  pd.read_csv("./raw_compiled_transcripts.csv", index_col = "Participant_ID")['PHQ_Binary']

X_train, X_test, y_train, y_test = train_test(X, y, testfile)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Undersample

In [ ]:
RANDOM_STATE = 42

# because the RandomUnderSampler does not work here
def undersampling(X_train, y_train):
    random.seed(RANDOM_STATE)
    
    neg_list = [i for i in range(len(y_train)) if y_train[i] == 0]
    pos_list = [i for i in range(len(y_train)) if y_train[i] == 1]
    
    if len(neg_list) < len(pos_list):
        minority_list = neg_list
        majority_list = pos_list
    else:
        minority_list = pos_list
        majority_list = neg_list
        
    sampled_list = random.sample(majority_list, len(minority_list))
    
    final_list = sampled_list + minority_list
    
    X_train_us = []
    y_train_us = []
    
    for i in final_list:
        X_train_us.append(X_train[i])
        y_train_us.append(y_train[i])
                                 
    return np.array(X_train_us), np.array(y_train_us)

xu_train, yu_train = undersampling(X_train, y_train)

In [ ]:
# shuffle the train data in unison because data is in order
# reduces poor performance during k-cross validation when sampling data
from sklearn.utils import shuffle

xu_train, yu_train = shuffle(X_train, y_train, random_state=RANDOM_STATE)

xu_train.shape, X_test.shape, yu_train.shape, y_test.shape


# Evaluation Metrics

In [ ]:
def evaluate_on_training_set(y_test, y_pred):
    print(y_pred)
    print(y_test)
    
    # Calculate AUC
    print("AUC is: ", roc_auc_score(y_test, y_pred))

    # print out recall and precision
    print(classification_report(y_test, y_pred))

    # print out confusion matrix
    print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

    # # calculate points for ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)

    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc_score(y_test, y_pred))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')

In [ ]:
def k_cross(input_model, X=X_train, y=y_train, k=10, n=1, random_state=RANDOM_STATE):
    f1_scores = []
    recall_scores = []
    rkf = RepeatedKFold(n_splits=k, n_repeats=n, random_state=RANDOM_STATE)
        
    for train_index, val_index in rkf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
                
        model = clone(input_model) # prevents incremental fitting
        model.fit(X_train, y_train) 
        
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred)
        f1_scores.append(f1)
        recall = recall_score(y_val, y_pred)
        recall_scores.append(recall)
        
    return f1_scores, recall_scores

## Evaluate Model 

In [ ]:
def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    evaluate_on_training_set(y_test, y_pred)
    
    return model

# Models

## Logistic Regression 

In [ ]:
def find_best_logreg_model(power):
    best_f1_model = None
    best_f1 = -1
    best_recall = -1
    
    for i in range(power + 1):
        model = LogisticRegression(n_jobs=3, C=10**i)
        
        f1_scores, recall_scores = k_cross(model)
        f1 = np.mean(f1_scores)
        recall = np.mean(recall_scores)

        if f1 > best_f1:
            best_f1 = f1
            best_recall = recall
            best_f1_model = model
    
    print(f"best scores: f1 = {best_f1}, recall = {best_recall}")
    
    return best_f1_model

best_logreg_f1_model = find_best_logreg_model(6)
evaluate_model(best_logreg_f1_model)

## Decision Tree

In [ ]:
def find_best_tree_model(upper_depth, upper_leaf):
    best_f1_model = None
    best_f1 = -1
    best_recall = -1
    
    for depth in range(1, upper_depth + 1):
        for leaf in range(1, upper_leaf + 1):
            model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth, min_samples_leaf=leaf) 
            
            f1_scores, recall_scores = k_cross(model)
            f1 = np.mean(f1_scores)
            recall = np.mean(recall_scores)
            
            if f1 > best_f1:
                best_f1 = f1
                best_recall = recall
                best_f1_model = model
    
    print(f"best scores: f1 = {best_f1}, recall = {best_recall}")
    
    return best_f1_model

best_tree_f1_model = find_best_tree_model(20, 30)
evaluate_model(best_tree_f1_model)

## Random Forest 

In [ ]:
def find_best_forest_model(n_estimators):
    best_f1_model = None
    best_f1 = -1
    best_recall = -1
    
    for estimator in range(1, n_estimators + 1):
        model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=estimator) 
        
        f1_scores, recall_scores = k_cross(model)
        f1 = np.mean(f1_scores)
        recall = np.mean(recall_scores)

        if f1 > best_f1:
            best_f1 = f1
            best_recall = recall
            best_f1_model = model
    
    print(f"best scores: f1 = {best_f1}, recall = {best_recall}")
    
    return best_f1_model

best_forest_f1_model = find_best_forest_model(30)
evaluate_model(best_forest_f1_model)

## SVM with Grid Search 

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100]},
                    {'kernel': ['poly'], 'degree': [3, 4, 5], 'C': [1, 10, 100]},
                    {'kernel': ['linear'], 'C': [1, 10, 100]}]

svm_model_cv = GridSearchCV(SVC(), tuned_parameters, cv=10, scoring='f1', verbose=1, n_jobs=4)
  -
f1_scores, recall_scores = k_cross(svm_model_cv)

print(f"f1 mean score: {np.mean(f1_scores)}")
print(f"recall mean score: {np.mean(recall_scores)}")

best_svm_model = evaluate_model(svm_model_cv)
print(f"Best SVM parameters: {best_svm_model.best_params_}")